# Ответы на вопросы в заданиях

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display
from pandas.api.types import CategoricalDtype

# in order not to overload we will 
# put all the auxiliary scripts in a separate file
import helpers


In [2]:
# df = pd.read_csv('https://cms.skillfactory.ru/assets/courseware/v1/c00e440dce1926e20bcf7b959cbeebbb/asset-v1:Skillfactory+DST-WEEKLY-2.0+08JULY2020+type@asset+block/main_task.csv')
df = pd.read_csv('./datasets/main_task.csv')

display(
    df.info()
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


None

In [3]:
df.columns = [
    'restaurant_id',
    'city',
    'cuisine_style',
    'ranking',
    'rating',
    'price_range',
    'number_of_reviews',
    'reviews',
    'url_ta',
    'id_ta'
]
df.head(3)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781


## `price_range` —  диапазон цен в ресторане

###   4.1 Вопросы о ценах

- Сколько вариантов непустых значений встречается в столбце Price Range?
- Сколько ресторанов относятся к среднему ценовому сегменту?

In [4]:
df.price_range.value_counts(dropna=False)

$$ - $$$    18412
NaN         13886
$            6279
$$$$         1423
Name: price_range, dtype: int64

Преобразуем данные в колонках в числовую категорию и избавимся от знаков \$ (служебный символ). Пропущенные значения обозначим как среднюю категорию (самое часто встречающееся значение)

In [5]:
price_range_map = {
    np.NaN: 2,
    '$': 1,
    '$$ - $$$': 2,
    '$$$$': 3
}

df.price_range = df.price_range.replace(to_replace=price_range_map)

df['price_range'] = df.price_range.astype(
    CategoricalDtype(categories=[1, 2, 3], ordered=True)
)

df.price_range.value_counts(dropna=False)

2    32298
1     6279
3     1423
Name: price_range, dtype: int64

## `city` —  город, в котором находится ресторан

### 4.2 Вопрос о городах

- Сколько городов представлено в наборе данных?

In [6]:
df.city.isnull().any()

False

In [7]:
len(df.city.value_counts())

31

### `Cuisine Style` — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;

### 4.3 Вопросы о кухнях

 - Сколько типов кухонь представлено в наборе данных?

In [8]:
# Преобразуем в список (list) строку с типами кухонь

df['cuisine_style'] = df.cuisine_style.apply(helpers.convert_cuisine)
total_cuisine = df['cuisine_style'].explode('cuisine_style')
len(total_cuisine.value_counts())


125

- Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.

In [9]:
total_cuisine.value_counts().head(1).index[0]

'Vegetarian Friendly'

- Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [10]:
# создадим новую колонку (feature engeniring)
df['total_types_cuisine'] = df.cuisine_style.fillna(1).apply(lambda x: 1 if isinstance(x, int) else len(x))
df['total_types_cuisine'].mean().round(1)

2.6

###  `Reviews` — данные о двух отзывах, которые отображаются на сайте ресторана;

### 4.4 Вопросы об отзывах

- Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.

In [11]:
reviews_df = pd.DataFrame(
    data=list(df.reviews.apply(helpers.convert_reviews)), 
    columns=['review1', 'review_date1', 'review2', 'review_date2']
)
reviews_df['review_date1'] = reviews_df.review_date1.apply(helpers.convert_datetime)
reviews_df['review_date2'] = reviews_df.review_date2.apply(helpers.convert_datetime)

df = pd.concat([df, reviews_df], axis=1)
df.drop(['reviews'], axis=1, inplace=True) 

In [12]:
df.head(3)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,url_ta,id_ta,total_types_cuisine,review1,review_date1,review2,review_date2
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,2,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,Good food at your doorstep,2017-12-31,A good hotel restaurant,2017-11-20
1,id_1535,Stockholm,NaN,1537.0,4.0,2,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,Unique cuisine,2017-07-06,Delicious Nepalese food,2016-06-19
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,Catch up with friends,2018-01-08,Not exceptional,2018-01-06


In [13]:
df.review_date1.max(), df.review_date2.max()

(Timestamp('2018-02-26 00:00:00'), Timestamp('2018-02-26 00:00:00'))

In [16]:
df['timedelta_reviews'] = df.review_date1 - df.review_date2

df.head(3)

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,url_ta,id_ta,total_types_cuisine,review1,review_date1,review2,review_date2,timedelta_reviews
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,2,194.0,/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,Good food at your doorstep,2017-12-31,A good hotel restaurant,2017-11-20,41 days
1,id_1535,Stockholm,NaN,1537.0,4.0,2,10.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,Unique cuisine,2017-07-06,Delicious Nepalese food,2016-06-19,382 days
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,3,688.0,/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,Catch up with friends,2018-01-08,Not exceptional,2018-01-06,2 days


In [18]:
df.timedelta_reviews.max()

Timedelta('3207 days 00:00:00')

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype          
---  ------               --------------  -----          
 0   restaurant_id        40000 non-null  object         
 1   city                 40000 non-null  object         
 2   cuisine_style        30717 non-null  object         
 3   ranking              40000 non-null  float64        
 4   rating               40000 non-null  float64        
 5   price_range          40000 non-null  category       
 6   number_of_reviews    37457 non-null  float64        
 7   url_ta               40000 non-null  object         
 8   id_ta                40000 non-null  object         
 9   total_types_cuisine  40000 non-null  int64          
 10  review1              33528 non-null  object         
 11  review_date1         33529 non-null  datetime64[ns] 
 12  review2              28972 non-null  object         
 13  review_date2    